In [51]:
import re
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import requests
import json

In [52]:
suspended_df = pd.read_csv("NYSE", sep = ";", header = None)
suspended_df.columns = ["Issuer Name", "Symbol", "Effective Date"]

In [53]:
ticker = df.to_dict('index')

In [54]:
ticker

{0: {'Issuer Name': 'AGFiQ U.S. Market Neutral Size Fund',
  'Symbol': 'SIZ',
  'Effective Date': '2019-02-20'},
 1: {'Issuer Name': 'Amira Nature Foods, Ltd.',
  'Symbol': 'ANFI',
  'Effective Date': '2018-11-16'},
 2: {'Issuer Name': 'Babcock & Wilcox Enterprises, Inc.',
  'Symbol': 'BW',
  'Effective Date': '2018-11-27'},
 3: {'Issuer Name': 'Bellatrix Exploration Ltd.',
  'Symbol': 'BXE',
  'Effective Date': '2018-07-23'},
 4: {'Issuer Name': 'BioPharmX Corporation',
  'Symbol': 'BPMX',
  'Effective Date': '2018-08-27'},
 5: {'Issuer Name': 'Braskem S.A.',
  'Symbol': 'BAK',
  'Effective Date': '2018-05-16'},
 6: {'Issuer Name': 'CSOP MSCI China A International Hedged ETF',
  'Symbol': 'CNHX',
  'Effective Date': '2018-08-15'},
 7: {'Issuer Name': 'Camber Energy, Inc.',
  'Symbol': 'CEI',
  'Effective Date': '2017-08-03'},
 8: {'Issuer Name': 'Capitol Acquisition Corp. IV',
  'Symbol': 'CIC',
  'Effective Date': '2018-10-03'},
 9: {'Issuer Name': 'China Green Agriculture, Inc.',
  

In [55]:
def build_query_list():
    query_list0 = []
    query_list1 = []
    for i in ticker:
        comp = re.sub(r'[,\.]',' ',ticker[i]['Issuer Name']).split()
        str_comp = '+'.join(comp)
        str_comp1 = ' '.join(comp)
        comp.append(ticker[i]['Symbol'])
        query = '+'.join(comp)
        query1 = ' '.join(comp)
        query_list0.append(query)
        query_list1.append(query1)
    return (query_list0, query_list1)

In [56]:
(query_list0, query_list1) = build_query_list()

In [57]:
# return top 1 results
def parseSingleNews(query):
    url = 'https://news.google.com/rss/search?q='+query+'+stock+suspend&hl=en-US&gl=US&ceid=US:en'
    response = urllib.request.urlopen(url)  
    soup = BeautifulSoup(response, 'html.parser')
    
    channel = soup.find('channel')
    news_list = []
    for item in soup.findAll('item')[0:1]:
        des = ''
        abst = ''
        description = item.find('description').get_text()
        match = re.search(r'(href=")(.*)(" target)',description)
        match1 = re.search(r'(<p>)(.*)(</p>)',description)
        if (match):
            des = match.group(2)
        if (match1):
            abst = match1.group(2)
        news_item = {
            'title':item.find('title').get_text(),
            'pubdate':item.find('pubdate').get_text(),
            'link':des,
            'abstract':abst
        }
        news_list.append(news_item)
    return news_list

In [58]:
def allQuery(query_list):
    dict = {}
    for q in query_list:
        q1 = q.replace('+',' ')
        news_list = parseSingleNews(q)
        dict[q1]=news_list
    return dict

In [59]:
google_news = allQuery(query_list0)

In [60]:
query_list2 = []
for q in query_list1:
    qry = {'keyword':q}
    query_list2.append(qry)

In [61]:
# return top 1 results
def parseSingleNews1(query):
    r = requests.get('https://globenewswire.com/Search', params=query)
    result = BeautifulSoup(r.text,'html.parser')
    contents = result.find_all("div", attrs = {"class" : "results-link"})
    news_list = []
    for content in contents[0:1]: 
        a = content.find("h1", attrs = {"class" : "post-title16px"}).find("a")
        title = a.text
        pubdate = content.find("span",attrs = {"class" : "dt-green"}).get_text()
        link = a['href']
        abstract = content.find_all("p")[1].text
        news_item = {
            'title':title,
            'pubdate':pubdate,
            'link':'https://globenewswire.com/'+link,
            'abstract':abstract
        }
        news_list.append(news_item)
    return news_list

In [62]:
def allQuery1(query_list):
    dict = {}
    for q in query_list:
        news_list = parseSingleNews1(q)
        dict[q['keyword']]=news_list
    return dict

In [63]:
ss = parseSingleNews1(query_list2[0])

In [64]:
globe_news = allQuery1(query_list2)

In [65]:
all_news = {}
for key in google_news:
    all_news[key] = google_news[key]
    all_news[key] += globe_news[key]

In [66]:
for k,v in all_news.items():
    print("Company name: {}".format(k))
    for i in v:
        print("\tRetrieved result:\t{}".format(i))
    print("------------------------------------\n")

Company name: AGFiQ U S Market Neutral Size Fund SIZ
------------------------------------

Company name: Amira Nature Foods Ltd ANFI
	Retrieved result:	{'title': 'Amira Nature Foods: Does Apparent Debt Default Mean Equity Is A Zero? - Seeking Alpha', 'pubdate': 'Wed, 28 Dec 2016 08:00:00 GMT', 'link': 'https://seekingalpha.com/article/4033020-amira-nature-foods-apparent-debt-default-mean-equity-zero', 'abstract': 'Amira appears to have defaulted on its bank debt, but has failed to disclose this in its SEC filings. Recent disclosures by ANFI appear to indicate that its ope.'}
	Retrieved result:	{'title': 'UPDATE -- ICR Announces Presenting Companies for Its 20th Annual Conference', 'pubdate': 'November 20, 2017', 'link': 'https://globenewswire.com//news-release/2017/11/20/1197759/0/en/UPDATE-ICR-Announces-Presenting-Companies-for-Its-20th-Annual-Conference.html', 'abstract': 'NEW YORK, Nov.  20, 2017  (GLOBE NEWSWIRE) -- ICR, a leading strategic communications and advisory firm serving 